In [6]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from pandas import Series, DataFrame
import seaborn as sns
from scipy import stats
from efficient_apriori import apriori

In [7]:
#read data
df = pd.read_csv("./heart.csv", sep=",")

In [83]:
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [99]:
features = df.drop(columns=['sex', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall'])
apdf = df.drop(columns=['exng', 'thalachh', 'oldpeak'])

In [100]:
apdf['age'] = pd.cut(apdf.age,bins=[0,2,17,65,99],labels=['Toddler/Baby','Child','Adult','Elderly'])
apdf['trtbps'] = pd.cut(apdf.trtbps,bins=[0,110,140,250],labels=['LowBP','NormalBP','HighBP'])
apdf['chol'] = pd.cut(apdf.chol,bins=[180,220, 240, 450],labels=['normal chl','high chl','very high chl'])

In [101]:
apdf = apdf.astype('category')
apdf['sex'] = apdf['sex'].cat.rename_categories({ 0 : 'male', 1 : 'female'})
apdf['cp'] = apdf['cp'].cat.rename_categories({ 0 : 'cp1', 1 : 'cp2', 2 : 'cp3', 3 : 'cp4'})
apdf['fbs'] = apdf['fbs'].cat.rename_categories({ 0 : '< 120 mg/dl', 1 : '> 120 mg/dl'})
apdf['restecg'] = apdf['restecg'].cat.rename_categories({ 0 : 'normal', 1 : 'ST-T wave abnormality', 2: 'left ventricular hypertrophy'})
apdf['slp'] = apdf['slp'].cat.rename_categories({ 0 : 'slp0', 1 : 'slp1', 2: 'slp2', 3: 'slp3'})
apdf['caa'] = apdf['caa'].cat.rename_categories({ 0 : 'caa1', 1 : 'caa2', 2: 'caa3'})
apdf['thall'] = apdf['thall'].cat.rename_categories({ 0 : 'thall0', 1 : 'thall1', 2: 'thall2', 3: 'thall3'})
apdf['output'] = apdf['output'].cat.rename_categories({ 0 : 'heart attack', 1 : 'no heart attack'})

In [171]:
apdf['age'].value_counts()

Adult           270
Elderly          33
Toddler/Baby      0
Child             0
Name: age, dtype: int64

In [102]:
apdf

,age,sex,cp,trtbps,chol,fbs,restecg,slp,caa,thall,output
0,Adult,female,cp4,HighBP,high chl,> 120 mg/dl,normal,slp0,caa1,thall1,no heart attack
1,Adult,female,cp3,NormalBP,very high chl,< 120 mg/dl,ST-T wave abnormality,slp0,caa1,thall2,no heart attack
2,Adult,male,cp2,NormalBP,normal chl,< 120 mg/dl,normal,slp2,caa1,thall2,no heart attack
3,Adult,female,cp2,NormalBP,high chl,< 120 mg/dl,ST-T wave abnormality,slp2,caa1,thall2,no heart attack
4,Adult,male,cp1,NormalBP,very high chl,< 120 mg/dl,ST-T wave abnormality,slp2,caa1,thall2,no heart attack
...,...,...,...,...,...,...,...,...,...,...,...
298,Adult,male,cp1,NormalBP,very high chl,< 120 mg/dl,ST-T wave abnormality,slp1,caa1,thall3,heart attack
299,Adult,female,cp4,LowBP,very high chl,< 120 mg/dl,ST-T wave abnormality,slp1,caa1,thall3,heart attack
300,Elderly,female,cp1,HighBP,normal chl,> 120 mg/dl,ST-T wave abnormality,slp1,caa3,thall3,heart attack
301,Adult,female,cp1,NormalBP,NaN,< 120 mg/dl,ST-T wave abnormality,slp1,caa2,thall3,heart attack


In [189]:
apdf1 = apdf.drop(columns=['age'])
transactions_from_apdf = [tuple(row) for row in apdf1.values.tolist()]
itemsets, rules = apriori(transactions_from_apdf, min_support=.3, min_confidence=.8, output_transaction_ids=True)
rules

[{NormalBP} -> {< 120 mg/dl},
 {ST-T wave abnormality} -> {< 120 mg/dl},
 {caa1} -> {< 120 mg/dl},
 {cp1} -> {< 120 mg/dl},
 {female} -> {< 120 mg/dl},
 {heart attack} -> {< 120 mg/dl},
 {no heart attack} -> {< 120 mg/dl},
 {normal} -> {< 120 mg/dl},
 {slp1} -> {< 120 mg/dl},
 {slp2} -> {< 120 mg/dl},
 {thall2} -> {< 120 mg/dl},
 {thall3} -> {< 120 mg/dl},
 {very high chl} -> {< 120 mg/dl},
 {heart attack} -> {female},
 {thall3} -> {female},
 {NormalBP, ST-T wave abnormality} -> {< 120 mg/dl},
 {NormalBP, caa1} -> {< 120 mg/dl},
 {NormalBP, female} -> {< 120 mg/dl},
 {NormalBP, no heart attack} -> {< 120 mg/dl},
 {NormalBP, thall2} -> {< 120 mg/dl},
 {caa1, female} -> {< 120 mg/dl},
 {caa1, no heart attack} -> {< 120 mg/dl},
 {< 120 mg/dl, no heart attack} -> {caa1},
 {caa1, thall2} -> {< 120 mg/dl},
 {cp1, female} -> {< 120 mg/dl},
 {female, heart attack} -> {< 120 mg/dl},
 {< 120 mg/dl, heart attack} -> {female},
 {no heart attack, slp2} -> {< 120 mg/dl},
 {no heart attack, thall2} -

In [181]:
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule)

{caa1, slp2} -> {< 120 mg/dl} (conf: 0.901, supp: 0.271, lift: 1.058, conv: 1.502)
{NormalBP, ST-T wave abnormality} -> {< 120 mg/dl} (conf: 0.903, supp: 0.307, lift: 1.060, conv: 1.530)
{caa1, thall2} -> {< 120 mg/dl} (conf: 0.912, supp: 0.343, lift: 1.071, conv: 1.693)
{caa1, slp1} -> {< 120 mg/dl} (conf: 0.913, supp: 0.208, lift: 1.072, conv: 1.708)
{male, no heart attack} -> {< 120 mg/dl} (conf: 0.917, supp: 0.218, lift: 1.077, conv: 1.782)
{ST-T wave abnormality, thall2} -> {< 120 mg/dl} (conf: 0.919, supp: 0.261, lift: 1.079, conv: 1.825)
{caa1, cp1} -> {< 120 mg/dl} (conf: 0.938, supp: 0.201, lift: 1.102, conv: 2.413)
{male, no heart attack} -> {thall2} (conf: 0.958, supp: 0.228, lift: 1.749, conv: 10.851)
{cp1, thall3} -> {heart attack} (conf: 0.910, supp: 0.234, lift: 1.999, conv: 6.068)


In [190]:
rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule)

{normal} -> {< 120 mg/dl} (conf: 0.823, supp: 0.399, lift: 0.967, conv: 0.840)
{thall3} -> {< 120 mg/dl} (conf: 0.838, supp: 0.323, lift: 0.984, conv: 0.915)
{very high chl} -> {< 120 mg/dl} (conf: 0.840, supp: 0.416, lift: 0.987, conv: 0.928)
{female} -> {< 120 mg/dl} (conf: 0.841, supp: 0.574, lift: 0.987, conv: 0.932)
{heart attack} -> {< 120 mg/dl} (conf: 0.841, supp: 0.383, lift: 0.987, conv: 0.932)
{slp2} -> {< 120 mg/dl} (conf: 0.859, supp: 0.403, lift: 1.009, conv: 1.054)
{no heart attack} -> {< 120 mg/dl} (conf: 0.861, supp: 0.469, lift: 1.011, conv: 1.065)
{slp1} -> {< 120 mg/dl} (conf: 0.864, supp: 0.399, lift: 1.015, conv: 1.094)
{NormalBP} -> {< 120 mg/dl} (conf: 0.864, supp: 0.568, lift: 1.015, conv: 1.095)
{cp1} -> {< 120 mg/dl} (conf: 0.874, supp: 0.413, lift: 1.027, conv: 1.180)
{ST-T wave abnormality} -> {< 120 mg/dl} (conf: 0.875, supp: 0.439, lift: 1.028, conv: 1.188)
{thall2} -> {< 120 mg/dl} (conf: 0.880, supp: 0.482, lift: 1.033, conv: 1.233)
{caa1} -> {< 120 mg/